In [1]:
import pandas as pd
import numpy as np
import os
import json
from PIL import Image
import matplotlib.pyplot as plt
from biovlp.utils.constants import CHEXPERT_COMPETITION_TASKS, CHEXPERT_TASKS, CHEXPERT_UNCERTAIN_MAPPINGS

In [2]:
mimic_dir = "/home/r15user2/Documents/CXR_dataset/mimic_data/2.0.0"
mimic_img_dir = "/home/r15user2/Documents/CXR_dataset/mimic_data"
master_df = pd.read_csv(os.path.join(mimic_dir, "master.csv"))
# keep one image one study
# master_df.drop_duplicates(subset=["subject_id", "study_id"], inplace=True)

chexpert_df = pd.read_csv(os.path.join(mimic_dir, "mimic-cxr-2.0.0-chexpert.csv"))
chexpert_df.fillna(0, inplace=True)

master_df["report"] = master_df["impression"] + " " + master_df["findings"]

In [3]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232421 entries, 0 to 232420
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Path          232421 non-null  object 
 1   dicom_id      232421 non-null  object 
 2   subject_id    232421 non-null  int64  
 3   study_id      232421 non-null  object 
 4   ViewPosition  232421 non-null  object 
 5   StudyDate     232421 non-null  int64  
 6   StudyTime     232421 non-null  float64
 7   impression    232421 non-null  object 
 8   findings      232421 non-null  object 
 9   split         232421 non-null  object 
 10  report        232421 non-null  object 
dtypes: float64(1), int64(2), object(8)
memory usage: 19.5+ MB


In [4]:
master_df["split"].value_counts()

split
train    227459
test       3082
valid      1880
Name: count, dtype: int64

In [5]:
# # only use training set
# split = "train"
# split_df = master_df[master_df["split"] == split]
# split_df["imgpath"] = split_df["Path"].apply(lambda x: os.path.join(mimic_img_dir, x))
# split_df["study_id"] = split_df["study_id"].apply(lambda x: int(x[1:]))

master_df["study_id"] = master_df["study_id"].apply(lambda x: int(x[1:]))
merged_df = pd.merge(master_df, chexpert_df, how="left", on=["subject_id", "study_id"])

In [6]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232421 entries, 0 to 232420
Data columns (total 25 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Path                        232421 non-null  object 
 1   dicom_id                    232421 non-null  object 
 2   subject_id                  232421 non-null  int64  
 3   study_id                    232421 non-null  int64  
 4   ViewPosition                232421 non-null  object 
 5   StudyDate                   232421 non-null  int64  
 6   StudyTime                   232421 non-null  float64
 7   impression                  232421 non-null  object 
 8   findings                    232421 non-null  object 
 9   split                       232421 non-null  object 
 10  report                      232421 non-null  object 
 11  Atelectasis                 232421 non-null  float64
 12  Cardiomegaly                232421 non-null  float64
 13  Consolidation 

In [7]:
np.random.seed(42)

train_df = merged_df.loc[merged_df["split"] == "train"]
print(merged_df.shape)

task_dfs = []
for i, t in enumerate(CHEXPERT_COMPETITION_TASKS):
    index = np.zeros(14)
    index[i] = 1
    df_task = train_df[
        (train_df["Atelectasis"] == index[0])
        & (train_df["Cardiomegaly"] == index[1])
        & (train_df["Consolidation"] == index[2])
        & (train_df["Edema"] == index[3])
        & (train_df["Pleural Effusion"] == index[4])
        & (train_df["Enlarged Cardiomediastinum"] == index[5])
        & (train_df["Lung Lesion"] == index[7])
        & (train_df["Lung Opacity"] == index[8])
        & (train_df["Pneumonia"] == index[9])
        & (train_df["Pneumothorax"] == index[10])
        & (train_df["Pleural Other"] == index[11])
        & (train_df["Fracture"] == index[12])
        & (train_df["Support Devices"] == index[13])
    ]
    df_task = df_task.sample(n=200, random_state=42)
    task_dfs.append(df_task)
df_200 = pd.concat(task_dfs)

mimic_df_200 = df_200[["subject_id", "study_id", "dicom_id", "ViewPosition", "report"] + CHEXPERT_TASKS]
mimic_df_200.to_csv("mimic-cxr-5x200-val-meta.csv", index=False)

merged_df = merged_df[~merged_df["Path"].isin(df_200["Path"])]
print(merged_df.shape)

(232421, 25)
(231421, 25)


In [8]:
annotation_file = "/home/r15user2/Documents/CXR_dataset/knowledge_graph/mimic_annotation.json"

with open(annotation_file) as f:
    all_examples = json.load(f)

In [9]:
len(all_examples["test"])

3858

In [10]:
merged_df = merged_df.loc[:, ["dicom_id", "ViewPosition", "StudyDate", "StudyTime", "split"] + CHEXPERT_TASKS]

In [11]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 231421 entries, 0 to 232420
Data columns (total 19 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   dicom_id                    231421 non-null  object 
 1   ViewPosition                231421 non-null  object 
 2   StudyDate                   231421 non-null  int64  
 3   StudyTime                   231421 non-null  float64
 4   split                       231421 non-null  object 
 5   No Finding                  231421 non-null  float64
 6   Enlarged Cardiomediastinum  231421 non-null  float64
 7   Cardiomegaly                231421 non-null  float64
 8   Lung Lesion                 231421 non-null  float64
 9   Lung Opacity                231421 non-null  float64
 10  Edema                       231421 non-null  float64
 11  Consolidation               231421 non-null  float64
 12  Pneumonia                   231421 non-null  float64
 13  Atelectasis        

In [12]:
merged_df["split"].value_counts()

split
train    226459
test       3082
valid      1880
Name: count, dtype: int64

In [13]:
merged_df["split"] = merged_df["split"].replace({"valid": "val"})

In [15]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 231421 entries, 0 to 232420
Data columns (total 19 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   dicom_id                    231421 non-null  object 
 1   ViewPosition                231421 non-null  object 
 2   StudyDate                   231421 non-null  int64  
 3   StudyTime                   231421 non-null  float64
 4   split                       231421 non-null  object 
 5   No Finding                  231421 non-null  float64
 6   Enlarged Cardiomediastinum  231421 non-null  float64
 7   Cardiomegaly                231421 non-null  float64
 8   Lung Lesion                 231421 non-null  float64
 9   Lung Opacity                231421 non-null  float64
 10  Edema                       231421 non-null  float64
 11  Consolidation               231421 non-null  float64
 12  Pneumonia                   231421 non-null  float64
 13  Atelectasis        

In [30]:
new_dataset_ds = dict()
for split in ["train", "val", "test"]:
    print(split)
    
    examples = all_examples[split]
    print(len(examples))
    dataset_as_dfs = pd.DataFrame(examples)

    split_df = merged_df[merged_df["split"] == split]
    split_df.drop(columns=["split"], inplace=True)

    # merge json with metadata
    df = pd.merge(dataset_as_dfs, split_df, left_on="id", right_on="dicom_id", how="left")
    # only keep frontal view
    frontal_df = df.loc[df["ViewPosition"].isin(["PA", "AP"])]
    
    frontal_df.drop(columns=["dicom_id"], inplace=True)
    frontal_df.drop_duplicates(subset=["subject_id", "study_id", "report"], inplace=True)
    frontal_df.sort_values(by=["subject_id", "StudyDate", "StudyTime"], inplace=True)
    frontal_df.reset_index(drop=True, inplace=True)

    new_dataset_ds[split] = frontal_df.to_dict("records")
    print(len(frontal_df))
    

train
270790


/tmp/ipykernel_1381311/1487327455.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  split_df.drop(columns=["split"], inplace=True)
/tmp/ipykernel_1381311/1487327455.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frontal_df.drop(columns=["dicom_id"], inplace=True)
/tmp/ipykernel_1381311/1487327455.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frontal_df.drop_duplicates(subset=["subject_id", "study_id", "report"], inplace

145471
val
2130
1151
test
3858
2210


/tmp/ipykernel_1381311/1487327455.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  split_df.drop(columns=["split"], inplace=True)
/tmp/ipykernel_1381311/1487327455.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frontal_df.drop(columns=["dicom_id"], inplace=True)
/tmp/ipykernel_1381311/1487327455.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frontal_df.drop_duplicates(subset=["subject_id", "study_id", "report"], inplace

In [31]:
with open("/home/r15user2/Documents/CXR_dataset/temporal_CXR/mimic_annotation.json", "w") as f:
    json.dump(new_dataset_ds, f)

In [32]:
with open("/home/r15user2/Documents/CXR_dataset/temporal_CXR/mimic_annotation.json") as f:
    new_data = json.load(f)

In [33]:
len(new_data["train"])

145471

In [34]:
len(new_data["val"])

1151

In [35]:
len(new_data["test"])

2210

In [37]:
145471 + 1151 + 2210

148832